In [1]:
import pandas as pd
import numpy as np
import glob
import os
import warnings
warnings.filterwarnings("ignore")

### Step 1 Preprocess the text Level

In [2]:
reference_text_df = pd.read_csv("scraping_data/Targeted_Ouput/0_raw_metadata.csv")
reference_text_df = reference_text_df.rename(columns={"raw_text": "reference_text"})

df_text = pd.read_csv("scraping_data/Targeted_Ouput/0_raw_audio_rawtxt.csv")

In [3]:
def topic_label_conditions(s, col = 'video_file'):
    if "ant" in s[col].lower() :
        return "insect_ant"
    elif "beetle" in s[col].lower() :
        return "insect_beetle"
    elif "bug" in s[col].lower() :
        return "insect_beetle"
    elif ("peacock" in s[col].lower()) & ("bird" in s[col].lower()) :
        return "bird_peacock"
    elif ("peafowl" in s[col].lower()) & ("bird" in s[col].lower()) :
        return "bird_peacock"
    elif ("peacock" in s[col].lower())  :
        return "bird_peacock"
    elif ("peafowl" in s[col].lower())  :
        return "bird_peacock"
    elif ("flamingo" in s[col].lower())  & ("bird" in s[col].lower()) :
        return "bird_flamingo"
    elif "flamingo" in s[col].lower() :
        return "bird_flamingo"
    elif "tailorbird" in s[col].lower() :
        return "bird_tailorbird"
    else:
        return "bird_tailorbird"

In [4]:
df_text['main_topics'] = df_text.apply(topic_label_conditions, axis=1)
df_text['video_file'] = df_text['video_file'].str.strip().str.replace('  ', ' ').str.replace('   ', ' ').str.replace(' ', '_').str.replace('__', '_').str.replace("&", "")


# link_df['video_name'] = link_df['video_name'].str.strip().str.replace('  ', ' ').str.replace('   ', ' ').str.replace(' ', '_').str.replace('__', '_').str.replace("&", "")


# df_text_2 = df_text.merge(link_df, how = 'left', left_on = 'video_file', right_on = 'video_name' )

reference_text_df['main_topics'] = reference_text_df.apply(lambda x: topic_label_conditions(x, col = "meta_file"), axis=1)

df_merged = df_text.merge(reference_text_df, how='left', on='main_topics' )
df_merged = df_merged.loc[:, ~df_merged.columns.isin(['video_name', 'meta_file']) ]
df_merged = df_merged.drop(['Unnamed: 0'], axis = 1)
df_merged.head()

,video_file,raw_txt,main_topics,reference_text
0,insects_intro_ant_v,Did you find an ant hill. The app that you see...,insect_ant,['Ants are one of the most common insects that...
1,birds_intro_tailor_bird_v,This is a girl named masha. She's sleeping and...,bird_tailorbird,"['Tailorbird', 'Tailorbird- The Tailorbird has..."
2,birds_intro_peacock_v,Wow look at those feathers. The bird you found...,bird_peacock,"['PEAFOWL', 'Peacock is the National bird of I..."
3,insects_intro_beetle_v,Beetles have a hard shell like armor. And i sa...,insect_beetle,['We see many insects around us. Some have win...
4,birds_intro_flamingo_v,Let's learn about flamingos flamingos are tall...,bird_flamingo,"['Flamingo', 'Flamingo is about 110 to 150 cm ..."


In [5]:
df_merged.to_csv("scraping_data/Targeted_Ouput/text_level_df.csv", index = False)

### Step 2 Preprocess the Sentences Level

In [6]:
# pip install --upgrade numpy pip install --upgrade numpy

In [7]:
from Python_Functions.NLP_Data_Engineer_Functions import stop_words, text_cleaning, generate_words_level_timstamps,aggregate_sentences_timestamps


In [8]:
timestamp_texts_sentences = pd.read_csv("scraping_data/Targeted_Ouput/0_raw_audio_word_timestamp.csv")
timestamp_texts_sentences['Label'] = timestamp_texts_sentences.apply(lambda x: topic_label_conditions(x, col = "audio_source"), axis=1)

# replace all NAN value with empty space ""
timestamp_texts_sentences = timestamp_texts_sentences.fillna('')

timestamp_texts_sentences.head()

,video_file,raw_txt,start sec,end sec,audio_source,Label
0,0_insects_intro_ant_v,,0,10,insects_intro_ant_v,insect_ant
1,0_insects_intro_ant_v,,0,10,insects_intro_ant_v,insect_ant
2,1_insects_intro_ant_v,Did you find an ant hill.,10,20,insects_intro_ant_v,insect_ant
3,2_insects_intro_ant_v,Go to aldi's.,20,30,insects_intro_ant_v,insect_ant
4,3_insects_intro_ant_v,More about that. The app that you see outside ...,30,40,insects_intro_ant_v,insect_ant


In [9]:
# loop this functions to each audio file timestamped extracted clips
words_timestamps_df = pd.DataFrame()
splited_clip_level = list(timestamp_texts_sentences.video_file)
for i in splited_clip_level:
    
    words_timestamps_df_temp = generate_words_level_timstamps(clip_nm  = i, timestamp_texts_sentences_sub_df = timestamp_texts_sentences )
    words_timestamps_df = pd.concat([words_timestamps_df,words_timestamps_df_temp ], axis = 0)

words_timestamps_df = words_timestamps_df.reset_index(drop = True)

# save to csv
words_timestamps_df.to_csv("scraping_data/Targeted_Ouput/1_words_timestamps_df.csv", index = None)



In [10]:
audio_texts = pd.read_csv("scraping_data/Targeted_Ouput/0_raw_audio_rawtxt.csv")
audio_texts.head()

,Unnamed: 0,video_file,raw_txt
0,0,insects_intro_ant_v,Did you find an ant hill. The app that you see...
1,0,birds_intro_tailor_bird_v,This is a girl named masha. She's sleeping and...
2,0,birds_intro_peacock_v,Wow look at those feathers. The bird you found...
3,0,insects_intro_beetle_v,Beetles have a hard shell like armor. And i sa...
4,0,birds_intro_flamingo_v,Let's learn about flamingos flamingos are tall...


In [11]:
# loop the function above to genrate aggreagted time range for each tokenzied sentecnes
audio_sentences_timestamp_df = pd.DataFrame()
sources = list(audio_texts.video_file)
for i in range(0,len(sources) ):
    
    audio_sentences_timestamp_temp = aggregate_sentences_timestamps(txt = audio_texts["raw_txt"][i] , words_timestamps_df_temp = words_timestamps_df[words_timestamps_df['audio_source']==sources[i]].reset_index(drop=True), audio_source = sources[i] )
    if audio_sentences_timestamp_temp.empty:
        continue
    else: 
        audio_sentences_timestamp_df = pd.concat([audio_sentences_timestamp_df, audio_sentences_timestamp_temp], axis = 0)


------------------
300
416
-116
------------------
296
278
18
------------------
218
206
12
------------------
76
89
-13
------------------
340
336
4


In [12]:
# sentecnes level dataset merge with the reference text
audio_sentences_timestamp_df['info_loss_perc'] = np.around(np.abs(audio_sentences_timestamp_df['diff_ratio'] - 1) * 100, 3)

audio_sentences_timestamp_df['main_topics'] = audio_sentences_timestamp_df.apply(lambda x: topic_label_conditions(x, col = "audio_source"), axis=1)
df_merged_sentences_timestamp = audio_sentences_timestamp_df.merge(reference_text_df, how='left', on='main_topics' )


In [13]:
df_merged_sentences_timestamp.columns

Index(['tokenized_sentences', 'sentence_level_timstamp_min_sec',
       'sentence_level_timstamp_max_sec', 'sentence_level_timstamp_min_minute',
       'sentence_level_timstamp_max_minute', 'audio_source', 'diff_ratio',
       'info_loss_perc', 'main_topics', 'meta_file', 'reference_text'],
      dtype='object')

In [14]:
df_merged_sentences_timestamp.head()

,tokenized_sentences,sentence_level_timstamp_min_sec,sentence_level_timstamp_max_sec,sentence_level_timstamp_min_minute,sentence_level_timstamp_max_minute,audio_source,diff_ratio,info_loss_perc,main_topics,meta_file,reference_text
0,did you find an ant hill,11.666667,20.0,0.1944,0.3333,insects_intro_ant_v,0.721154,27.885,insect_ant,insects_intro_ant_t,['Ants are one of the most common insects that...
1,the app that you see outside or only a small n...,23.333333,40.0,0.3889,0.6667,insects_intro_ant_v,0.721154,27.885,insect_ant,insects_intro_ant_t,['Ants are one of the most common insects that...
2,these huge groups of ants are called calm he s...,40.476190,60.0,0.6746,1.0000,insects_intro_ant_v,0.721154,27.885,insect_ant,insects_intro_ant_t,['Ants are one of the most common insects that...
3,all of the worker ants all females,59.411765,62.0,0.9902,1.0333,insects_intro_ant_v,0.721154,27.885,insect_ant,insects_intro_ant_t,['Ants are one of the most common insects that...
4,the meal ask job is the fertilized the queen t...,62.400000,71.5,1.0400,1.1917,insects_intro_ant_v,0.721154,27.885,insect_ant,insects_intro_ant_t,['Ants are one of the most common insects that...


In [15]:
df_merged_sentences_timestamp.to_csv("scraping_data/Targeted_Ouput/1_audio_sentences_timestamp_df.csv", index = None)



### Step 3 Spelling Correction 

In [16]:
# pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-2.2.0/en_core_web_sm-2.2.0.tar.gz

In [17]:
# pip install spaCy==2.2

In [18]:
from platform import python_version

print(python_version())

3.8.13


In [19]:
# pip install --upgrade ipykernel

In [20]:
import en_core_web_sm
import spacy
nlp = en_core_web_sm.load()

naturallanguageprocessing = spacy.load('en')

In [21]:
import pandas as pd
import numpy as np
from Python_Functions.NLP_Data_Engineer_Functions import text_cleaning, stop_words
import nltk
from nltk.tokenize import sent_tokenize

import spacy
from gramformer import Gramformer
import torch
import en_core_web_sm
from nltk.tokenize import word_tokenize

nlp = en_core_web_sm.load()
nlp = en_core_web_sm.load()

In [22]:
nlp = spacy.load("en")
# nlp = spacy.load("en_core_web_sm")

In [23]:
# pip install git+https://github.com/PrithivirajDamodaran/Gramformer.git

In [24]:
from Python_Functions.NLP_spelling_correction import levenshtein, \
        UncommonWords, word_cnt_compare, Correct_Sentences, version_select, set_seed

In [25]:
sentences_extracted_pd =  pd.read_csv("scraping_data/Targeted_Ouput/1_audio_sentences_timestamp_df.csv")
sentences_extracted_pd.head()

,tokenized_sentences,sentence_level_timstamp_min_sec,sentence_level_timstamp_max_sec,sentence_level_timstamp_min_minute,sentence_level_timstamp_max_minute,audio_source,diff_ratio,info_loss_perc,main_topics,meta_file,reference_text
0,did you find an ant hill,11.666667,20.0,0.1944,0.3333,insects_intro_ant_v,0.721154,27.885,insect_ant,insects_intro_ant_t,['Ants are one of the most common insects that...
1,the app that you see outside or only a small n...,23.333333,40.0,0.3889,0.6667,insects_intro_ant_v,0.721154,27.885,insect_ant,insects_intro_ant_t,['Ants are one of the most common insects that...
2,these huge groups of ants are called calm he s...,40.476190,60.0,0.6746,1.0000,insects_intro_ant_v,0.721154,27.885,insect_ant,insects_intro_ant_t,['Ants are one of the most common insects that...
3,all of the worker ants all females,59.411765,62.0,0.9902,1.0333,insects_intro_ant_v,0.721154,27.885,insect_ant,insects_intro_ant_t,['Ants are one of the most common insects that...
4,the meal ask job is the fertilized the queen t...,62.400000,71.5,1.0400,1.1917,insects_intro_ant_v,0.721154,27.885,insect_ant,insects_intro_ant_t,['Ants are one of the most common insects that...


In [26]:
text_extracted_pd = pd.read_csv("scraping_data/Targeted_Ouput/text_level_df.csv")
videos_files_lst = list(text_extracted_pd['video_file'])
text_extracted_pd.head()

,video_file,raw_txt,main_topics,reference_text
0,insects_intro_ant_v,Did you find an ant hill. The app that you see...,insect_ant,['Ants are one of the most common insects that...
1,birds_intro_tailor_bird_v,This is a girl named masha. She's sleeping and...,bird_tailorbird,"['Tailorbird', 'Tailorbird- The Tailorbird has..."
2,birds_intro_peacock_v,Wow look at those feathers. The bird you found...,bird_peacock,"['PEAFOWL', 'Peacock is the National bird of I..."
3,insects_intro_beetle_v,Beetles have a hard shell like armor. And i sa...,insect_beetle,['We see many insects around us. Some have win...
4,birds_intro_flamingo_v,Let's learn about flamingos flamingos are tall...,bird_flamingo,"['Flamingo', 'Flamingo is about 110 to 150 cm ..."


In [27]:


files_identifiers = list(text_extracted_pd['video_file'].unique())

temp_txt = text_extracted_pd[text_extracted_pd['video_file'] == 'birds_intro_flamingo_v']['raw_txt'].values[0]
temp_text = temp_txt.replace(".", "")
sentences = sent_tokenize(temp_txt)
sentences_no_periods = sent_tokenize(temp_text )



In [28]:
# implement Gramformer API to correct text sentences 
def Correct_Sentences(file_identifier, temp_txt):
    
#     nlp = spacy.load("en_core_web_sm")
    nlp = en_core_web_sm.load()
#     set_seed(1212)
    gf = Gramformer(models = 1, use_gpu=False) # 1=corrector, 2=detector
    
    sentences = sent_tokenize(temp_txt)
    
    raw_sentences =  sentences 
    corrected_sentences = []
    Levenshtain_scores = []
    Jaccard_scores = []
    words_diff = []
    for i in range(0, len(raw_sentences)):
        raw_sentence = raw_sentences[i]
        corrected_sentence = list(gf.correct(raw_sentence, max_candidates=1))[0]
        corrected_sentences.append(corrected_sentence )
        Levenshtain_scores.append(levenshtein(seq1 = raw_sentence , seq2 = corrected_sentence))
        Jaccard_scores.append(nltk.jaccard_distance(set(raw_sentence), set(corrected_sentence) ))
        words_diff.append(word_cnt_compare(raw_sentence, corrected_sentence))
        
    # formulatae the corrected sentences back to txt chunk
    start_txt = ''
    for indx, sen in enumerate(corrected_sentences):
        if indx == 0:
            corrected_txt = start_txt + sen 
        else:
            corrected_txt = corrected_txt + " " + sen
    
    # form dataframe from all the outputs
    res_df = pd.DataFrame()
    res_df["file_name"] = [file_identifier] * len(corrected_sentences)
    res_df["raw_txt"] = [temp_txt] * len(corrected_sentences)
    res_df["raw_sentences"] = raw_sentences
    res_df["corrected_sentences"] = corrected_sentences
    res_df["corrected_txt"] = corrected_txt
    res_df["sentences_levenshtain_scores"] = Levenshtain_scores
    res_df["sentences_jaccard_scores"] = Jaccard_scores
    res_df["sentences_num_words_diff"] = words_diff
    res_df["sentences_num_words_diff_abs"] = np.abs(words_diff)
    
    raw_sentences_word_counts = [len(word_tokenize(i)) for i in list(res_df["raw_sentences"])]
    res_df["raw_sentences_word_counts"] = raw_sentences_word_counts
    res_df["correction_ratio"] = res_df["sentences_num_words_diff_abs"]/res_df["raw_sentences_word_counts"]
    

    # select to keep the corewcted txt vs keep the raw_txt
    res_df['final_corrected_version_sentences_txt'] = res_df.apply(lambda x: version_select(x.raw_sentences, x.corrected_sentences, x.correction_ratio, x.raw_sentences_word_counts ), axis=1)
    
    # formulatae the final corrected sentences back to txt chunk
    start_txt = ''
    for indx, sen in enumerate(list(res_df['final_corrected_version_sentences_txt'])):
        if indx == 0:
            final_txt = start_txt + sen 
        else:
            final_txt = corrected_txt + " " + sen
    
    res_df['final_corrected_version_sentences_txt'] = final_txt
    
    return res_df

In [29]:
files_identifiers = list(text_extracted_pd['video_file'].unique())

In [30]:
nlp = spacy.load('en')

In [31]:
# import spacy.cli 
# spacy.cli.download("en_core_web_md")

In [32]:
# pip list

In [33]:
result_corrected_df = pd.DataFrame()
for file in files_identifiers:
    
    temp_txt = text_extracted_pd[text_extracted_pd['video_file'] == file]['raw_txt'].values[0]
    temp_pd = Correct_Sentences(file_identifier = file, temp_txt = temp_txt)
    result_corrected_df = pd.concat([result_corrected_df, temp_pd ], axis = 0)
    
result_corrected_df.reset_index(drop=True)

/Users/mikewu/opt/anaconda3/envs/python3.8/lib/python3.8/site-packages/tensorflow/python/framework/dtypes.py:513: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  np.object,
/Users/mikewu/opt/anaconda3/envs/python3.8/lib/python3.8/site-packages/tensorflow/python/framework/dtypes.py:521: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  np.bool,
/Users/mikewu/opt/anaconda3/envs/python3.8/lib/python3.8/site-packages/ten

[Gramformer] Grammar error correct/highlight model loaded..
[Gramformer] Grammar error correct/highlight model loaded..
[Gramformer] Grammar error correct/highlight model loaded..
[Gramformer] Grammar error correct/highlight model loaded..
[Gramformer] Grammar error correct/highlight model loaded..


,file_name,raw_txt,raw_sentences,corrected_sentences,corrected_txt,sentences_levenshtain_scores,sentences_jaccard_scores,sentences_num_words_diff,sentences_num_words_diff_abs,raw_sentences_word_counts,correction_ratio,final_corrected_version_sentences_txt
0,insects_intro_ant_v,Did you find an ant hill. The app that you see...,Did you find an ant hill.,Did you find an ant hill?,Did you find an ant hill? The app that you see...,1.0,0.133333,0,0,7,0.000000,Did you find an ant hill? The app that you see...
1,insects_intro_ant_v,Did you find an ant hill. The app that you see...,The app that you see outside or only a small n...,"The app that you see outside, or only a small ...",Did you find an ant hill? The app that you see...,1.0,0.045455,-1,1,20,0.050000,Did you find an ant hill? The app that you see...
2,insects_intro_ant_v,Did you find an ant hill. The app that you see...,These huge groups of ants are called calm he's...,These huge groups of ants are called calm he i...,Did you find an ant hill? The app that you see...,20.0,0.107143,3,3,54,0.055556,Did you find an ant hill? The app that you see...
3,insects_intro_ant_v,Did you find an ant hill. The app that you see...,All of the worker ants all females.,All of the workers were all females.,Did you find an ant hill? The app that you see...,5.0,0.062500,0,0,8,0.000000,Did you find an ant hill? The app that you see...
4,insects_intro_ant_v,Did you find an ant hill. The app that you see...,The meal ask job is the fertilized the queen t...,The meal ask job is the fertilized queen they ...,Did you find an ant hill? The app that you see...,4.0,0.000000,1,1,24,0.041667,Did you find an ant hill? The app that you see...
...,...,...,...,...,...,...,...,...,...,...,...,...
120,birds_intro_flamingo_v,Let's learn about flamingos flamingos are tall...,Flamingos are actually very comfortable standi...,Flamingos are actually very comfortable standi...,Let’s learn about flamingos. flamingos are tal...,0.0,0.000000,0,0,10,0.000000,Let’s learn about flamingos. flamingos are tal...
121,birds_intro_flamingo_v,Let's learn about flamingos flamingos are tall...,Scientists think they took one leg up into the...,Scientists think they took one leg up into the...,Let’s learn about flamingos. flamingos are tal...,0.0,0.000000,0,0,15,0.000000,Let’s learn about flamingos. flamingos are tal...
122,birds_intro_flamingo_v,Let's learn about flamingos flamingos are tall...,And help them stay warm when the temperature o...,And help them stay warm when the temperature o...,Let’s learn about flamingos. flamingos are tal...,0.0,0.000000,0,0,18,0.000000,Let’s learn about flamingos. flamingos are tal...
123,birds_intro_flamingo_v,Let's learn about flamingos flamingos are tall...,I hope you enjoy learning about flamingos today.,I hope you enjoy learning about flamingos today.,Let’s learn about flamingos. flamingos are tal...,0.0,0.000000,0,0,9,0.000000,Let’s learn about flamingos. flamingos are tal...
